<a href="https://colab.research.google.com/github/sickopickle/Paperless/blob/main/Python_Bezier_curve_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import math
EPSILON = 1e-12
MACHINE_EPSILON = 1.12e-16


class point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
#NOTE calculations may be off because of int vs float
def isMachineZero (val) : 
  return (val >= -MACHINE_EPSILON and val <= MACHINE_EPSILON)
def hypot (x, y) : 
  return math.sqrt(x * x + y * y)
#def point (x, y) : return ( Point(x,y) )
def pointLength (p) : 
  return hypot(p.x, p.y)
def pointNegate (p) : 
  return point(-p.x, -p.y)
def pointAdd (p1, p2) : 
  return point(p1.x + p2.x, p1.y + p2.y)
def pointSubtract (p1, p2) : 
  return point(p1.x - p2.x, p1.y - p2.y)
def pointMultiplyScalar (p, n) : 
  return point(p.x * n, p.y * n)
def pointDot (p1, p2) : 
  return p1.x * p2.x + p1.y * p2.y
def pointDistance (p1, p2) : 
  return hypot(p1.x - p2.x, p1.y - p2.y)
def pointNormalize (p, length = 1) : 
  return pointMultiplyScalar(p, (length / pointLength(p)) if pointLength(p) else 0) #POSSIBLE FLAW
class createSegment: 
  def __init__(self, p, i, o):
    self.p=p
    self.i=i
    self.o=o #POSSIBLE FLAW (CHANGED)

def chordLengthParameterize (points, first, last)  :
    u = [0]
    for i in range(first+1, last+1):
    #for (let i = first + 1 i <= last i++) :
        #u[i - first] = u[i - first - 1] + pointDistance(points[i], points[i - 1])
        u.append(u[i - first - 1] + pointDistance(points[i], points[i - 1]))
    
    for i in range(1, last-first+1):
    #for (let i = 1, m = last - first i <= m i++) :
        u[i] /= u[last-first]
    
    return u

# Use Newton-Raphson iteration to find better root.
def findRoot (curve, point, u) :
    curve1 = []
    curve2 = []
    # Generate control vertices for Q'
    for i in range(3) : #FLAG
        curve1.append(pointMultiplyScalar(pointSubtract(curve[i + 1], curve[i]), 3))
    
    # Generate control vertices for Q''
    for i in range(2):
    #for (let i = 0 i <= 1 i++) :
        curve2.append(pointMultiplyScalar(pointSubtract(curve1[i + 1], curve1[i]), 2))
    
    # Compute Q(u), Q'(u) and Q''(u)
    pt = evaluate(3, curve, u)
    pt1 = evaluate(2, curve1, u)
    pt2 = evaluate(1, curve2, u)
    diff = pointSubtract(pt, point)
    df = pointDot(pt1, pt1) + pointDot(diff, pt2)
    # u = u - f(u) / f'(u)
    #return isMachineZero(df) ? u : u - pointDot(diff, pt1) / df
    return (u if isMachineZero(df) else (u-pointDot(diff,pt1)/df))

# Evaluate a bezier curve at a particular parameter value
def evaluate (degree, curve, t) :
    # Copy array
    tmp = curve[:]#.slice() FLAG
    # Triangle computation
    for i in range(1, degree+1):
    #for (let i = 1 i <= degree i++) :
        for j in range(degree):
        #for (let j = 0 j <= degree - i j++) :
            tmp[j] = pointAdd(pointMultiplyScalar(tmp[j], 1 - t), pointMultiplyScalar(tmp[j + 1], t))

    print("tmp" + str(tmp[0].x) + " " + str(tmp[0].y))
    return tmp[0]

def addCurve (segments, curve)  :
    prev = segments[len(segments) - 1]
    prev.o = pointSubtract(curve[1], curve[0])
    segments.append(createSegment(curve[3], pointSubtract(curve[2], curve[3]), None))

# Use least-squares method to find Bezier control points for region.
def generateBezier (points, first, last, uPrime, tan1, tan2)  :
    epsilon = EPSILON
    #abs = Math.abs
    pt1 = points[first]
    pt2 = points[last]
    # Create the C and X matrices
    C = [[0, 0],[0, 0]]
    X = [0, 0]
    for i in range(0, last-first+1):
    #for (let i = 0, l = last - first + 1 i < l i++) :
        u = uPrime[i] 
        t = 1 - u
        b = 3 * u * t
        b0 = t * t * t
        b1 = b * t
        b2 = b * u
        b3 = u * u * u
        a1 = pointNormalize(tan1, b1)
        a2 = pointNormalize(tan2, b2)
        #print(pt2)
        #print(type(pt1))
        tmp = pointSubtract(pointSubtract(points[first + i], pointMultiplyScalar(pt1, b0 + b1)), pointMultiplyScalar(pt2, b2 + b3))
        C[0][0] += pointDot(a1, a1)
        C[0][1] += pointDot(a1, a2)
        # C[1][0] += a1.dot(a2)
        C[1][0] = C[0][1]
        C[1][1] += pointDot(a2, a2)
        X[0] += pointDot(a1, tmp)
        X[1] += pointDot(a2, tmp)
    
    # Compute the determinants of C and X
    detC0C1 = C[0][0] * C[1][1] - C[1][0] * C[0][1]
    alpha1=None
    alpha2=None
    if (abs(detC0C1) > epsilon) :
        # Kramer's rule
        detC0X = C[0][0] * X[1] - C[1][0] * X[0]
        detXC1 = X[0] * C[1][1] - X[1] * C[0][1]
        # Derive alpha values
        alpha1 = detXC1 / detC0C1
        alpha2 = detC0X / detC0C1
    
    else :
        # Matrix is under-determined, try assuming alpha1 == alpha2
        c0 = C[0][0] + C[0][1]
        c1 = C[1][0] + C[1][1]
        #alpha1 = alpha2 = abs(c0) > epsilon ? X[0] / c0 : abs(c1) > epsilon ? X[1] / c1 : 0
        alpha1 = alpha2 = X[0] / c0 if abs(c0) > epsilon else X[1] / c1 if abs(c1) > epsilon else 0
    
    # If alpha negative, use the Wu/Barsky heuristic (see text)
    # (if alpha is 0, you get coincident control points that lead to
    # divide by zero in any subsequent NewtonRaphsonRootFind() call.
    segLength = pointDistance(pt2, pt1)
    eps = epsilon * segLength
    handle1 = None
    handle2 = None
    if (alpha1 < eps or alpha2 < eps) :
        # fall back on standard (probably inaccurate) formula,
        # and subdivide further if needed.
        alpha1 = alpha2 = segLength / 3
    
    else :
        # Check if the found control points are in the right order when
        # projected onto the line through pt1 and pt2.
        line = pointSubtract(pt2, pt1)
        # Control points 1 and 2 are positioned an alpha distance out
        # on the tangent vectors, left and right, respectively
        handle1 = pointNormalize(tan1, alpha1)
        handle2 = pointNormalize(tan2, alpha2)
        if (pointDot(handle1, line) - pointDot(handle2, line) > segLength * segLength) :
            # Fall back to the Wu/Barsky heuristic above.
            alpha1 = alpha2 = segLength / 3
            handle1 = handle2 = None # Force recalculation
        
    
    # First and last control points of the Bezier curve are
    # positioned exactly at the first and last data points
    return [pt1, pointAdd(pt1, handle1 or pointNormalize(tan1, alpha1)), pointAdd(pt2, handle2 or pointNormalize(tan2, alpha2)), pt2]

# Given set of points and their parameterization, try to find
# a better parameterization.
def reparameterize (points, first, last, u, curve) :
    #for (let i = first i <= last i++) :
    for i in range(first, last+1):
        u[i - first] = findRoot(curve, points[i], u[i - first])
    
    # Detect if the new parameterization has reordered the points.
    # In that case, we would fit the points of the path in the wrong order.
    #for (let i = 1, l = u.length i < l i++) :
    for i in range (1, len(u)):
        if (u[i] <= u[i - 1]):
            return False
    
    return True


class error:
  def __init__(self, error, index):
    self.error = error
    self.index = index
# Find the maximum squared distance of digitized points to fitted curve.
def findMaxError (points, first, last, curve, u)  :
    index = math.floor((last - first + 1) / 2)
    maxDist = 0
    #for (let i = first + 1 i < last i++) :
    for i in range (first+1, last):
        P = evaluate(3, curve, u[i - first])
        v = pointSubtract(P, points[i])
        dist = v.x * v.x + v.y * v.y # squared
        if (dist >= maxDist) :
            maxDist = dist
            index = i
        
    return error(maxDist, index)
    #return :
    #   error: maxDist,
    #    index: index,
    



def fit (points, closed, error) :
    if (closed) :
        points.insert(0, points[len(points) - 1])
        points.append(points[1]) 
    
    length = len(points)
    if length == 0 and type(length) == int:
        return []
    
    segments = [createSegment(points[0], None, None)]
    fitCubic(points, segments, error, 0, length - 1, 
    pointSubtract(points[1], points[0]), 
    pointSubtract(points[length - 2], points[length - 1]))
    if (closed) :
        segments.pop(0)
        segments.pop()
    
    return segments
#NOTE MAY HAVE TO FIND ALTERNATIVE TO === instead of using ==
def fitCubic (points, segments, error, first, last, tan1, tan2):
    #  Use heuristic if region only has two points in it
    if (last - first == 1 and type(last-first)==int) :
        pt1 = points[first]
        pt2 = points[last]
        dist = pointDistance(pt1, pt2) / 3
        addCurve(segments, [pt1, pointAdd(pt1, pointNormalize(tan1, dist)), pointAdd(pt2, pointNormalize(tan2, dist)), pt2])
        return
    
    # Parameterize points, and attempt to fit curve
    uPrime = chordLengthParameterize(points, first, last)
    maxError = max(error, error * error)
    split=None
    parametersInOrder = True
    # Try not 4 but 5 iterations
    #for (let i = 0 i <= 4 i++) :
    for i in range(5):
        curve = generateBezier(points, first, last, uPrime, tan1, tan2)
        #print("<curve")
        for point in curve:
          print(str(point.x)+ " " + str(point.y))
        #print("curve>")
        #  Find max deviation of points to fitted curve
        Max = findMaxError(points, first, last, curve, uPrime)
        print("Max " + str(Max.error) + " " + str(Max.index))
        if (Max.error < error and parametersInOrder) :
            addCurve(segments, curve)
            #print("<segments")
            #for segment in segments:
            #  print((str(segment.i.x) if segment.i else "None") + " " + (str(segment.i.y) if segment.i else "None") + " " + (str(segment.o.x) if segment.o else "None") + " " + (str(segment.o.y) if segment.o else "None") + " " + (str(segment.p.x) if segment.p else "None") + " " + (str(segment.p.y) if segment.p else "None"))
            #print("segments>")
            #print("<curve")
            #for point in curve:
            #  print(str(point.x)+" " + str(point.y))
            #print("curve>")
            return
        
        split = Max.index
        # If error not too large, try reparameterization and iteration
        if (Max.error >= maxError):
            break
        parametersInOrder = reparameterize(points, first, last, uPrime, curve)
        maxError = Max.error
    
    # Fitting failed -- split at max error point and fit recursively
    tanCenter = pointSubtract(points[split - 1], points[split + 1])
    fitCubic(points, segments, error, first, split, tan1, tanCenter)
    fitCubic(points, segments, error, split, last, pointNegate(tanCenter), tan2)



# Assign parameter values to digitized points
# using relative distances between points.



def getSegmentsPathData (segments, closed, precision)  :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    '''def addSegment (segment, skipLine=False)  : #NOTE
        curX = segment.p.x
        curY = segment.p.y
        global first
        if (first) :
            parts.append('M' + formatPair(curX, curY))
            first = False
        
        else :
            inX = curX + (segment.i.x if segment.i.y else 0)
            inY = curY + (segment.i.y if segment.i.y else 0)
            #NOTE if error occurs figure out optional chaining
            if (inX == curX and inY == curY and outX == prevX and outY == prevY) :
                # l = relative lineto:
                if not skipLine :
                    dx = curX - prevX
                    dy = curY - prevY
                    #parts.append(dx == 0 ? 'v' + round(dy) : dy == 0 ? 'h' + round(dx) : 'l' + formatPair(dx, dy))
                    parts.append('v' + Round(dy) if dx == 0 else ('h' + Round(dx) if dy==0 else 'l' + formatPair(dx, dy)))
                
            
            else :
                # c = relative curveto:
                parts.append('c' +
                    formatPair(outX - prevX, outY - prevY) +
                    ' ' +
                    formatPair(inX - prevX, inY - prevY) +
                    ' ' +
                    formatPair(curX - prevX, curY - prevY))
            
        
        prevX = curX
        prevY = curY
        #outX = curX + (segment.o?.x ?? 0)
        #outY = curY + (segment.o?.y ?? 0)
        outX = curX #+ (segment.o.x if segment.o.x else 0)
        outY = curY #+ (segment.o.y if segment.o.y else 0)'''
    
    if (not length):
        return ''
    for i in range(length):
    #for (let i = 0 i < length i++)
        #addSegment(segments[i])
        segment=segments[i]
        skipLine=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
            first = False
        
        else :
            inX = curX + (segment.i.x if segment.i.y else 0)
            inY = curY + (segment.i.y if segment.i.y else 0)
            #NOTE if error occurs figure out optional chaining
            if (inX == curX and inY == curY and outX == prevX and outY == prevY and type(inX) == type(curX) and type(inY) == type(curY) and type(outX) == type(prevX) and type(outY) == type(prevY)) :
                # l = relative lineto:
                if not skipLine :
                    dx = curX - prevX
                    dy = curY - prevY
                    #parts.append(dx == 0 ? 'v' + round(dy) : dy == 0 ? 'h' + round(dx) : 'l' + formatPair(dx, dy))
                    parts.append('v' + Round(dy, precisionMultiplier) if (dx == 0 and type(dx)==int) else ('h' + Round(dx, precisionMultiplier) if (dy==0 and type(dy)==int) else 'l' + formatPair(dx, dy)))
                
            
            else :
                # c = relative curveto:
                parts.append('c' +
                    formatPair(outX - prevX, outY - prevY) +
                    ' ' +
                    formatPair(inX - prevX, inY - prevY) +
                    ' ' +
                    formatPair(curX - prevX, curY - prevY))
            
        
        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        '''try:
          outX = curX+segment.o.x
        except (NameError, AttributeError) as e:
          outX = curX

        try:
          outY = curY+segment.o.y
        except (NameError, AttributeError) as e:
          outY = curY'''
    # Close path by drawing first segment again
    if (closed and length > 0) :
        #addSegment(segments[0], True)
        segment=segments[0]
        skipLine=True
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
            first = False
        
        else :
            inX = curX + (segment.i.x if segment.i.y else 0)
            inY = curY + (segment.i.y if segment.i.y else 0)
            #NOTE if error occurs figure out optional chaining
            if (inX == curX and inY == curY and outX == prevX and outY == prevY) :
                # l = relative lineto:
                if not skipLine :
                    dx = curX - prevX
                    dy = curY - prevY
                    #parts.append(dx == 0 ? 'v' + round(dy) : dy == 0 ? 'h' + round(dx) : 'l' + formatPair(dx, dy))
                    parts.append('v' + Round(dy, precisionMultiplier) if dx == 0 else ('h' + Round(dx, precisionMultiplier) if dy==0 else 'l' + formatPair(dx, dy)))
                
            
            else :
                # c = relative curveto:
                parts.append('c' +
                    formatPair(outX - prevX, outY - prevY) +
                    ' ' +
                    formatPair(inX - prevX, inY - prevY) +
                    ' ' +
                    formatPair(curX - prevX, curY - prevY))
            
        
        prevX = curX
        prevY = curY
        #outX = curX + (segment.o?.x ?? 0)
        #outY = curY + (segment.o?.y ?? 0)
        '''try:
          outX = curX+segment.o.x
        except (NameError, AttributeError) as e:
          outX = curX
        try:
          outY = curY+segment.o.y
        except (NameError, AttributeError) as e:
          outY = curY'''
        outX = curX + (segment.o.x if segment.o else 0)
        #print(segment.o.x)
        outY = curY + (segment.o.y if segment.o else 0)
        #print(segment.o.y)
        parts.append('z')
    
    return ''.join(parts)

class options:
  def __init__(self, closed, tolerance, precision):
    self.closed = closed
    self.tolerance = tolerance
    self.precision=precision

def f1(p):
  return point(p.x, p.y)

def f2(p):
  return point(p[0], p[1])

#def simplify(points, options) :
#  return fit(list(map((f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2), points)), options.closed, options.tolerance if options.tolerance else 2.5)

def simplifySvgPath (points, options)  :
    if (len(points) == 0 and type(len(points))==int) :
        return ''
    return getSegmentsPathData(fit(list(map(f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)
    #return getSegmentsPathData(fit(points.map(typeof points[0].x == 'number' ? (p) => point(p.x, p.y) : (p) => point(p[0], p[1])), options.closed, options.tolerance ?? 2.5), options.closed, options.precision ?? 5)

In [4]:
def arrayToPoints (arr) :
  points = []
  for i in arr:
    points.append(point(i[0], i[1]))
  return(points)

In [14]:
print(simplifySvgPath(arrayToPoints([[75, 46], [68, 47], [68, 47], [65, 50], [61, 61], [56, 82], [56, 103], [60, 119], [66, 131], [68, 134]]), options(False, 15, 0)))

75 46
42.938931584524944 50.58015263078215
55.65281999464413 115.4792299919662
68 134
tmp68.80564267041692 47.862411796861835
tmp68.80564267041692 47.862411796861835
tmp65.70438251537666 49.767258268702506
tmp59.342326619257314 57.61361961908818
tmp54.80476802826708 79.10588300790681
tmp57.094724253139916 103.4311149372426
tmp61.83499557371951 120.89529419179118
tmp66.65330576104115 131.79991658349158
Max 14.215453121447347 4
M75.0,46.0c-32.06107,4.58015 -19.34718,69.47923 -7.0,88.0
